In [9]:
import pandas as pd
import ndjson
import ijson
import os
import praw
import multiprocessing

In [4]:
dataset = pd.read_csv('dataset/dataset.csv', header=0)

In [5]:
subreddits = dataset[['subreddit_id','subreddit']]
subreddits.drop_duplicates(inplace=True)
print(len(subreddits))
subreddits.head()

857541


/usr/lib/python3/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,subreddit_id,subreddit
0,t5_n9nn1,Market76
1,t5_2qo4s,nba
2,t5_31mhn,SakuraGakuin
3,t5_2qhyq,classicalmusic
4,t5_2dar36,China_Flu


In [6]:
def process_part(subreddits):
    reddit = praw.Reddit(client_id="2CkE8z4bPgIkhQ",
                     client_secret="fFIJX43pXxx8LvSQH4KAXMlZoXM",
                     user_agent="python:it.ppndrd.scraper:v1 (by u/ppndrd)")
    
    subs_dict = {}
    subs_dict["subreddit_id"] = []
    subs_dict["subreddit"] = []
    subs_dict["created_utc"] = []
    subs_dict["description"] = []
    subs_dict["over18"] = []
    subs_dict["subscribers"] = []

    for sub in subreddits.itertuples():
        name = sub[2]
        _id = sub[1]
        subreddit = reddit.subreddit(name)

        try:
            subreddit.id

            subs_dict["subreddit_id"].append(_id)
            subs_dict["subreddit"].append(name)
            subs_dict["created_utc"].append(subreddit.created_utc)
            subs_dict["description"].append(subreddit.description)
            subs_dict["over18"].append(subreddit.over18)
            subs_dict["subscribers"].append(subreddit.subscribers)

        except:
            pass
        
    return pd.DataFrame.from_dict(subs_dict)

In [7]:
inputs = []
n = 86
inputs.append(subreddits[:10000])
for i in range(1,n-1):
    inputs.append(subreddits[i*10000:(i+1)*10000])
inputs.append(subreddits[(n-1)*10000:])

In [ ]:
outputs = []
p = multiprocessing.Pool(multiprocessing.cpu_count())
outputs = p.map(process_part, inputs)

In [16]:
subreddits = pd.concat(outputs)
subreddits.to_csv('dataset/subreddits.csv', index=False)

In [14]:
print('fatto')

fatto


In [17]:
len(subreddits)

849558